In [1]:
##run this first to install package, then delete
conda install webdriver-manager selenium pandas

SyntaxError: invalid syntax (1152895909.py, line 2)

In [2]:
# import libraries
import time 
import json
import random
import requests
import pandas as pd 
from selenium import webdriver 
from selenium.webdriver import Chrome 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait

Run this block below as is

In [3]:
# start by defining the options 
options = webdriver.ChromeOptions() 
options.headless = True # it's more scalable to work in headless mode 
# normally, selenium waits for all resources to download 
# we don't need it as the page also populated with the running javascript code. 
options.page_load_strategy = 'none' 
# this returns the path web driver downloaded 
chrome_path = ChromeDriverManager().install() 
chrome_service = Service(chrome_path) 
# pass the defined options and service objects to initialize the web driver 
driver = Chrome(options=options, service=chrome_service) 
driver.implicitly_wait(5)

/var/folders/vv/c90tdsjn63g4803dfq4s7w4m0000gn/T/ipykernel_68579/210964506.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True # it's more scalable to work in headless mode


This stores a list of listing urls based on a desired quantity (each listing page contains 120 listings). I'm keeping the number low for now, just for efficiency but we'll have to discuss how many individual listings we want to try and scrape. 

In [4]:
url_list = []
page = 0
num_pages = 77

# get urls
for x in range(num_pages):
    page += 1 
    url_list.append("https://losangeles.craigslist.org/search/apa?query=los%20angeles#search=1~list~{}~0".format(page))

For each page of listings, we need to grab the urls for each of the 120 individual listings.

In [5]:
listings = []

for url in url_list:
    # create WebElement of Listings Page
    driver.get(url)
    
    driver.implicitly_wait(5)
    
    # find the section the listings are located in
    content = driver.find_element(By.CSS_SELECTOR, "div[class*='cl-results-page'")
    
    # find the section where the link to the property page is located for each 120 listings
    for element in content.find_elements(By.TAG_NAME, "li")[1:]:
        try:
            listings.append(element.find_elements(By.CSS_SELECTOR, "div[class*='result-node-wide']>a")[0].get_attribute("href"))
            
        except (StaleElementReferenceException, NoSuchElementException) as e:
            listings.append('NA')
    
            
    # add delay (not sure about actually length of time delays necessary)
    time.sleep(random.randint(1, 3))
    
listingsDf = pd.DataFrame(listings)
listingsDf.rename(columns = {0:"urls"}, inplace = True)
# limiting size for testing
listingsDf = listingsDf
# check DF
listingsDf.head()

,urls
0,https://losangeles.craigslist.org/wst/apa/d/lo...
1,https://losangeles.craigslist.org/wst/apa/d/lo...
2,https://losangeles.craigslist.org/wst/apa/d/lo...
3,https://losangeles.craigslist.org/lac/apa/d/lo...
4,https://losangeles.craigslist.org/lac/apa/d/lo...


Now we will scrape each webpage from the list of urls. This is where most of our development and discussion needs to occur. I've placed a temporary simple test in there for now. 

In [ ]:
listingsDf[listingsDf.urls == 'NA']

In [ ]:
listingsDf_trim = listingsDf[:1000]

In [ ]:
description = []
latitude = []
longitude = []
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)


for url in listingsDf_trim['urls']:
    # create WebElement of listing URL
    driver.get(url)
    
    driver.maximize_window() # For maximizing window
    driver.implicitly_wait(5) # gives an implicit wait for 20 seconds
   
    # grab description of listing and add to list
    try:
        #description
        description.append(WebDriverWait(driver, 5,ignored_exceptions=ignored_exceptions)\
                        .until(expected_conditions.presence_of_element_located((By.ID, "postingbody"))).text)
        
        #lat
        latitude_element = WebDriverWait(driver, 5, ignored_exceptions=ignored_exceptions).until(
                        expected_conditions.presence_of_element_located((By.ID, "map"))
                        )
        latitude_value = latitude_element.get_attribute("data-latitude")
        latitude.append(latitude_value)
                
        #long
        longitude_element = WebDriverWait(driver, 5, ignored_exceptions=ignored_exceptions).until(
                        expected_conditions.presence_of_element_located((By.ID, "map"))
                        )
        longitude_value = longitude_element.get_attribute("data-longitude")
        longitude.append(longitude_value)
        
    except (StaleElementReferenceException, NoSuchElementException, TimeoutException) as e:
        description.append('NA')
        latitude.append('NA')
        longitude.append('NA')
    
    time.sleep(random.randint(1, 3))
    
listingsDf_trim['description'] = description
listingsDf_trim['lat'] = latitude
listingsDf_trim['long'] = longitude

In [ ]:
listingsDf_trim

In [ ]:
listingsDf_trim_1000

In [ ]:
listingsDf_trim_100

In [ ]:
import re

def remove_and_clean(text):
    text = re.sub(("\n"), " ", text)
    text = text.lower()
    return text

listingsDf_trim['description'] = listingsDf_trim['description'].apply(remove_and_clean)

In [ ]:
import geopandas as gpd

parks = gpd.read_file('county_parks.geojson')
parks

In [ ]:
listingsDf_trim['park_TF'] = listingsDf_trim['description'].apply(lambda x: any(substring in x for substring in parks['PARK_NAME'].str.lower()))

<div id="map" class="viewposting leaflet-container leaflet-retina leaflet-fade-anim leaflet-grab leaflet-touch-drag" data-latitude="33.959712" data-longitude="-118.419410" data-accuracy="10" tabindex="0">

In [ ]:
listingsDf_trim.park_TF.value_counts()

In [ ]:
listingsDf_trim[listingsDf_trim['park_TF'] == True]

In [ ]:
listingsDf_trim = listingsDf_trim[listingsDf_trim['lat'] != 'NA']
test = gpd.GeoDataFrame(listingsDf_trim, geometry=gpd.points_from_xy(listingsDf_trim.long, listingsDf_trim.lat), crs="EPSG:4326")

In [ ]:
# import libraries
import plotly.express as px
import matplotlib.pyplot as plt


# create scatter map
fig = px.scatter_mapbox(test, lat=test.geometry.y, lon=test.geometry.x, color="park_TF",
                               mapbox_style="carto-positron",
                               #range_color=range_color,
                               zoom=9,
                               center = {"lat": 34, "lon": -118.4},
                               opacity=.8,

                               )

# options on the layout
fig.update_layout(
        width = 900,
        height = 700,
        title = "Listings",
        title_x = .5
    )
fig.show()

In [ ]:
original_df.to_pickle("./dummy.pkl")